In [ ]:
title = f"\nKEY RESERVOIR[Noise {ch_noise}][Eve {eve_presence}][Eve Threshold {eve_threshold}] "
simulation_parameters = f"\nNumber of keys : {num_keys},   Aimed length of keys : {KEY_LENGTH}  \nEve Presence : {eve_presence},   Channel Noise : {ch_noise},   Eve's threshold : {eve_threshold},   Precision of Error Measurement : {precision},   Error Threshold : {error_threshold} \nTime Taken : {elapsed_time}\n"

In [ ]:
avg_time_taken = np.zeros(num_keys, dtype = 'float32')
avg_QBERs = np.zeros(num_keys, dtype = 'float32')
avg_out_len = np.zeros(num_keys, dtype = 'float32')

med_Eve_detection = np.zeros(num_keys, dtype = 'float32')
med_Eve_Record = np.zeros(num_keys, dtype = 'float32')

for I in range(num_keys):
    for J in range(num_iter):
        avg_time_taken[I] += time_taken[J, I]   
        avg_out_len[I] += KEY_RESERVOIR_len[J, I]
        avg_QBERs[I] += QBERs[J, I]
        
    med_Eve_detection[I] = sum(Eve_detection[:, I]) >= num_iter/2 
    med_Eve_Record[I] = sum(Eve_Record[:, I]) >= num_iter/2
    
    avg_time_taken[I] /= num_iter   
    avg_out_len[I] /= num_iter
    avg_QBERs[I] /= num_iter

In [1]:
def get_ticks(data):
    mean = np.mean(data)
    Min = np.min(data)
    Max = np.max(data)
    # nmean = mean
    # while nmean < 10: nmean *= 10

    # sci_not = int(nmean/mean)   # To get till 1 point after decimal in scientific notation
    # low = int(0.7*nmean)
    # high = int(1.3*nmean)
    # ticks = np.arange(low, high, dec/20)/dec

    ticks = np.linspace(Min, Max, 10)
    return ticks

In [ ]:
incorrect = sum(sum(np.abs(Eve_detection - Eve_Record)))    #len(Eve_detection[J]) - sum(Eve_detection[J])
detection = f"Eve-detection was incorrect : {incorrect}/{sum(sum(Eve_detection))} times (= {incorrect/sum(sum(Eve_detection))} ); for QBER Threshold : {error_threshold}"

plt.figure(figsize=(7, 5))

iter_i = np.arange(num_iter)
df = pd.DataFrame({'indices' : indices, 'avg_time_taken' : avg_time_taken, 'med_Eve_detection' : med_Eve_Record}, index = indices )
mean_time = np.mean(avg_time_taken)

sns.scatterplot(x = indices, y = avg_time_taken, hue = med_Eve_Record)
plt.legend()
plt.yticks(np.arange(int(0.8*mean_time - 5), int(1.2*mean_time + 5)))
plt.xlabel('indices')
plt.ylabel('Avg Time Taken(in s)')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 8))

# For KEY_RESERVOIR_len
coefficients_len = np.polyfit(indices, avg_out_len, 1)
polynomial_len = np.poly1d(coefficients_len)
equation_len = f'y = {coefficients_len[0]:.2f}x + {coefficients_len[1]:.2f}'
ax[0, 0].text(0.05, 0.95, equation_len, transform=ax[0, 0].transAxes, fontsize=10, verticalalignment='top')
ax[0, 0].set_xlabel('Indices')
ax[0, 0].set_ylabel('Length of final key after Information Reconciliation')
ax[0, 0].set_title("Key length")
ax[0, 0].plot(indices, avg_out_len)
ax[0, 0].plot(indices, polynomial_len(indices), linestyle='--')
ax[0, 0].set_yticks(get_ticks(avg_out_len))
ax[0, 0].minorticks_on()
ax[0, 0].grid(True)

# For QBER
coefficients_qber = np.polyfit(indices, avg_QBERs, 1)
polynomial_qber = np.poly1d(coefficients_qber)
equation_qber = f'y = {coefficients_qber[0]:.2f}x + {coefficients_qber[1]:.2f}'
ax[0, 1].text(0.05, 0.95, equation_qber, transform=ax[0, 1].transAxes, fontsize=10, verticalalignment='top')
ax[0, 1].set_xlabel('Indices')
ax[0, 1].set_ylabel('Average QBER')
ax[0, 1].set_title("QBER")
ax[0, 1].plot(indices, avg_QBERs)
ax[0, 1].plot(indices, polynomial_qber(indices), linestyle='--')
ax[0, 1].set_yticks(get_ticks(avg_QBERs))
ax[0, 1].minorticks_on()
ax[0, 1].grid(True)

# For time taken for 1 cycle
coefficients_tt = np.polyfit(indices, avg_time_taken, 1) 
polynomial_tt = np.poly1d(coefficients_tt)
equation_tt = f'y = {coefficients_tt[0]:.2f}x + {coefficients_tt[1]:.2f}'
ax[1, 0].text(0.05, 0.95, equation_tt, transform=ax[1, 0].transAxes, fontsize=10, verticalalignment='top')
ax[1, 0].set_xlabel('Indices')
ax[1, 0].set_ylabel('Average Time taken(in seconds) for 1 cycle')
ax[1, 0].set_title("Time taken")
ax[1, 0].plot(indices, avg_time_taken)
ax[1, 0].plot(indices, polynomial_tt(indices), linestyle='--')
ax[1, 0].set_yticks(get_ticks(avg_time_taken))
ax[1, 0].minorticks_on()
ax[1, 0].grid(True)


# SKGR for keys of same length
coefficients_skgr = np.polyfit(iter_i, SKGR, 1)
polynomial_skgr = np.poly1d(coefficients_skgr)
equation_skgr = f'y = {coefficients_skgr[0]:.2f}x + {coefficients_skgr[1]:.2f}'
ax[1, 1].text(0.05, 0.95, equation_skgr, transform=ax[1, 0].transAxes, fontsize=10, verticalalignment='top')
ax[1, 1].set_xlabel('Iterations')
ax[1, 1].set_ylabel('SKGR(in seconds) for 1 cycle')
ax[1, 1].set_title("Time taken")
ax[1, 1].plot(iter_i, SKGR)
ax[1, 1].plot(iter_i, polynomial_skgr(iter_i), linestyle='--')
ax[1, 1].set_yticks(get_ticks(SKGR))
ax[1, 1].minorticks_on()
ax[1, 1].grid(True)

# # Fit the data to the hyperbolic function
# popt, pcov = curve_fit(hyperbolic_fit, iter_i, SKGR)
# a, b = popt
# # Generate the polynomial using the fitted parameters
# fitted_SKGR = hyperbolic_fit(iter_i , *popt)
# equation_SKGR = f'y = {a:.2f}/x + {b:.2f}'
# ax[1, 1].text(0.05, 0.95, equation_SKGR, transform=ax[1, 1].transAxes, fontsize=10, verticalalignment='top')
# ax[1, 1].set_xlabel('ITERATION')
# ax[1, 1].set_ylabel('Secure key generation rate')
# ax[1, 1].set_title("SKGR")
# ax[1, 1].plot(iter_i, SKGR, 'o', label='Data')  # Plot the original data
# ax[1, 1].plot(iter_i, fitted_SKGR, linestyle='--', label='Hyperbolic fit')  # Plot the hyperbolic fit
# ax[1, 1].minorticks_on()
# ax[1, 1].grid(True)


# Set the super title for the entire figure
plt.suptitle(title, weight='bold')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
print(title)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

ax[0].plot(indices, avg_out_len/1000, label = "avg Key length/1000")
ax[0].plot(indices, avg_QBERs, label = "avg QBERs")
ax[0].plot(indices, avg_time_taken, label = "avg time taken")
ax[0].set_xlabel('index')
ax[0].legend()

ax[1].plot(iter_i, SKGR, label = 'SKGR')
ax[1].set_yticks(get_ticks(SKGR))
ax[1].set_xlabel('iteration')
ax[1].legend()

plt.ylabel('counts')

plt.suptitle(f'All parameters in one for {title}')
# plt.legend()
plt.tight_layout()
print(f'All_parameters[Noise {ch_noise}]')

# Writing to the file 

In [ ]:
def write_ra() :
    filename = f"Data.txt"
    key_strings = f"Keys.txt"
    kdata_path = os.path.join(bb84_subdirectory, filename)
    key_path = os.path.join(bb84_subdirectory, key_strings)
    file = open(kdata_path, "a")
    file2 = open(key_path, "a")
    
    file.write(simulation_parameters)
    file2.write(simulation_parameters)

    
    file.write(f"\nInput Length = [{', '.join(map(str, in_len))}] \nAverage Time Taken = [{', '.join(map(str, avg_time_taken))}] \nAverage QBER = [{', '.join(map(str, avg_QBERs))}] \nAverage Output length = [{', '.join(map(str, avg_out_len))}]\n \nSKGR = [{', '.join(map(str, SKGR[J]))}]")
    file.close()
    
    file2.write(detection)
    file2.write(f" False detection indices : {False_detections}")
    file2.write(f"\nKeys = [{', '.join(map(str, keys))}] \n\n Eve detection = [{', '.join(map(str, Eve_detection))}]")
    file2.close()

    write = False
    print("Files updated")